In [ ]:
from llama_index.core import (
    VectorStoreIndex, 
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage
)
import os
from dotenv import load_dotenv
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
import json
from pydantic import BaseModel, Field, field_validator
from typing import List

load_dotenv()

True

### Vector Stores


In [7]:
VECTOR_STORE_DIR = 'chroma_db'
if not os.path.exists(VECTOR_STORE_DIR):
    os.makedirs(VECTOR_STORE_DIR)
    # Load documents
    documents = SimpleDirectoryReader("../Quick-Examples/data").load_data()
    # Verify that there are no empty documents
    documents = SimpleDirectoryReader("../Quick-Examples/data").load_data()
    for doc in documents:
        if not doc:
            print("Documento vacío encontrado")

    # Initialize the ChromaDB client
    db = chromadb.PersistentClient(path=VECTOR_STORE_DIR)

    # Create a new collection
    chroma_collection = db.get_or_create_collection('chroma_collection')

    # Assign chroma as the vector_store to the context
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    # Create index
    index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)
else:
    # Initialize the ChromaDB client
    db = chromadb.PersistentClient(path=VECTOR_STORE_DIR)

    # Get the collection
    chroma_collection = db.get_or_create_collection('chroma_collection')

    # Assign chroma as the vector_store to the context
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    # Load index from storage
    index = VectorStoreIndex.from_vector_store(
        vector_store, storage_context=storage_context
    )


In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.llms.google_genai import GoogleGenAI

llm_openai = OpenAI(model="gpt-4o-mini", api_key=os.environ["OPENAI_API_KEY"])
llm_gemini = GoogleGenAI(model="gemini-2.0-flash", api_key=os.environ["GEMINI_API_KEY"])

In [16]:
class ChapterExtraction(BaseModel):
    """Salida con los textos de cada capítulo y número del capítulo."""

    chapter_text: str = Field(..., description="El texto de cada capítulo del Tao Te Ching. (No incluye el número del capítulo)")
    topic: str = Field(
        ...,
        description="El tema de lo que trata el capítulo. Debe ser uno de los valores predeterminados: ['El Tao y la naturaleza del universo', 'La no-acción (Wu Wei)', 'Dualidad y equilibrio (Yin y Yang)', 'Simplicidad y humildad', 'Armonía social y liderazgo']",
    )

    @field_validator('topic')
    def validate_tema(cls, value):
        allowed_values = [
            'El Tao y la naturaleza del universo', 
            'La no-acción (Wu Wei)', 
            'Dualidad y equilibrio (Yin y Yang)', 
            'Simplicidad y humildad', 
            'Armonía social y liderazgo'
        ]
        if value not in allowed_values:
            raise ValueError(f"'{value}' no es un tema válido. Los temas permitidos son: {allowed_values}")
        return value
    
sllm_gemini = llm_gemini.as_structured_llm(output_cls=ChapterExtraction)

In [ ]:
from tqdm import tqdm
import time

query_engine = index.as_query_engine(
    similarity_top_k=50,
    llm=sllm_gemini,
    response_mode="tree_summarize",
)

capitulos = list(range(1, 81))
contenido_capitulos = {
    'chapter_text': [],
    'topic': []
}
for cap in tqdm(capitulos):
    response = query_engine.query(f"Qué dice el capítulo {cap} del Tao Te Ching?").response.model_dump()
    contenido_capitulos['chapter_text'].append(response['chapter_text'])
    contenido_capitulos['topic'].append(response['topic'])
    time.sleep(10)  # This assures that there is not 15 requests per minute (free tier API Key)

# guardar `contenido_capitulos` como un json
with open('contenido_capitulos.json', 'w') as f:
    json.dump(contenido_capitulos, f, ensure_ascii=False, indent=4)

In [18]:
from google import genai
import os
from google.genai import types

client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

SYSTEM_PROMPT = """"
Asume el rol de un erudito y practicante de la filosofía taoísta, inmerso en la escritura de un libro sobre el Tao Te Ching. Tu tarea es redactar un capítulo que explore y explique un versículo específico de esta obra atemporal.

Imagina que estás escribiendo para un lector que busca una comprensión profunda y práctica del Taoísmo. Tu enfoque debe ser accesible y reflexivo, guiando al lector a través de las complejidades del texto con claridad y sabiduría.

Comienza desentrañando las capas de significado inherentes al versículo elegido. Explora tanto su interpretación literal como las resonancias simbólicas que evoca dentro del contexto más amplio de la filosofía taoísta. Utiliza extractos selectos del versículo (sin necesidad de incluirlo completo) para fundamentar tus explicaciones, revelando cómo cada palabra y frase contribuye al mensaje central.

A continuación, profundiza en las implicaciones filosóficas del versículo. Conecta sus ideas con los principios fundamentales del Taoísmo, como el Wu Wei (no acción), la armonía con la naturaleza y la búsqueda del equilibrio. Reflexiona sobre cómo estas enseñanzas pueden transformar nuestra comprensión del mundo y nuestro lugar en él.

Trasciende la teoría y lleva la sabiduría del Tao Te Ching a la vida cotidiana. Ofrece ejemplos concretos de cómo los principios del versículo pueden aplicarse en situaciones personales, profesionales y sociales. Describe cómo la internalización de estas enseñanzas puede guiar nuestras decisiones y fomentar una existencia más auténtica y armoniosa.

Finalmente, ofrece una reflexión que capture la esencia del versículo y su relevancia para el lector contemporáneo. Destaca cómo la comprensión del Tao puede desbloquear una mayor paz interior, sabiduría y una profunda conexión con el universo

Estilo a tener en cuenta:

Enfoque en Narrativa: Centrarse en la redacción de un capítulo de libro, lo que fomenta un estilo más narrativo y fluido en lugar de un análisis estructurado en puntos.

Dividir en subsecciones: Dividir la reflexión en subsecciones, en función de los conceptos que se tratan en el capítulo del Tao especificado

Eliminación de Bullet Points: Eliminar explícitamente las referencias a esquemas o listas.

Lenguaje Evocador: Utilizar un lenguaje más descriptivo y evocador para inspirar una respuesta más creativa y reflexiva.

Énfasis en la Guía al Lector: Guiar al lector a través del texto con claridad y sabiduría.

Metáforas: Introducir metáforas
"""

USER_PROMPT = f"""
Interpreta este capítulo del Tao Te Ching: 
```
$/cap_text/$
```
"""

class InterpretationSchema(BaseModel):
    """Salida que contiene las secciones/capítulos de cada párrafo de la interpretación"""
    section: list[str] = Field(
        ...,
        description="Lista de secciones. Cada sección corresponde a un párrafo de la interpretación",
    )
    paragraph: list[str] = Field(
        ...,
        description="Lista de párrafos. Cada párrafo corresponde a una parte de la interpretación",
    )

# Cargar a un diccionario el json `../Quick-Examples/contenido_capitulos.json`
with open('../Quick-Examples/contenido_capitulos.json', 'r') as f:
    contenido_capitulos = json.load(f)

In [19]:
print(contenido_capitulos['chapter_text'][0])

El T ao que puede ser expresado con palabras 
no es el T ao eterno.
El nombre que puede ser pronunciado
no es el nombre eterno.
Lo que no tiene nombre es el principio del cielo y la tierra.
Lo que tiene nombre es la madre de todas las cosas.
La permanente ausencia de deseos
permite contemplar el gran misterio.
La constante presencia de deseos
permite contemplar sus manifestaciones.
Ambos estados tienen un origen común
y con nombres diferentes aluden a una misma realidad.
El infinito insondable es la puerta de todos los misterios.


In [42]:
contenido_capitulos['interpretation'] = []
for cap in tqdm(range(len(contenido_capitulos['chapter_text']))):

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=USER_PROMPT.replace("$/cap_text/$", contenido_capitulos['chapter_text'][cap]),
        config=types.GenerateContentConfig(
            system_instruction=SYSTEM_PROMPT,
            response_mime_type="application/json",
            response_schema=InterpretationSchema,
        ),
    )

    contenido_capitulos['interpretation'].append(json.loads(response.text))

    # This assures that there is not 15 requests per minute (free tier API Key)
    time.sleep(3)

# guardar `contenido_capitulos` como un json
with open('capitulos_con_interpretacion.json', 'w') as f:
    json.dump(contenido_capitulos, f, ensure_ascii=False, indent=4)

  0%|          | 0/80 [00:00<?, ?it/s]

100%|██████████| 80/80 [11:44<00:00,  8.80s/it]


In [20]:
with open('capitulos_con_interpretacion.json', 'r') as f:
    capitulos_con_interpretaciones = json.load(f)

In [51]:
capitulos_con_interpretaciones['chapter_text'][0]

'El T ao que puede ser expresado con palabras \nno es el T ao eterno.\nEl nombre que puede ser pronunciado\nno es el nombre eterno.\nLo que no tiene nombre es el principio del cielo y la tierra.\nLo que tiene nombre es la madre de todas las cosas.\nLa permanente ausencia de deseos\npermite contemplar el gran misterio.\nLa constante presencia de deseos\npermite contemplar sus manifestaciones.\nAmbos estados tienen un origen común\ny con nombres diferentes aluden a una misma realidad.\nEl infinito insondable es la puerta de todos los misterios.'

In [61]:
from markdown_pdf import MarkdownPdf, Section

pdf = MarkdownPdf(
    toc_level=3,
)

portada = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Antic+Didone&family=Forum&family=Nunito+Sans:ital,opsz,wght@0,6..12,200..1000;1,6..12,200..1000&family=Quicksand:wght@300..700&display=swap');
</style>

# Tao Te Ching
_**De Lao Tse**_
## Reflexiones e Interpretaciones

<div style="page-break-after: always;"></div>
"""

pdf.add_section(
    Section(
        text=portada
    )
)

for cap in range(len(capitulos_con_interpretaciones['chapter_text'])):
    css = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Antic+Didone&family=Forum&family=Nunito+Sans:ital,opsz,wght@0,6..12,200..1000;1,6..12,200..1000&family=Quicksand:wght@300..700&display=swap');

.chapter-text {
    width: 80%;
    margin: 0 auto;
    font-style: italic;
    line-height: 1.6;
    white-space: pre-wrap;  /* Preserva los saltos de línea */
}
</style>
"""
    markdown_text = css+f"""
# Capítulo {cap+1}

**{capitulos_con_interpretaciones['topic'][cap]}**

<div class="chapter-text">
{contenido_capitulos['chapter_text'][cap]}
</div>

<div style="page-break-after: always;"></div>

## Interpretación

"""
    # Agrega las secciones de interpretación directamente
    for sec, par in zip(capitulos_con_interpretaciones['interpretation'][cap]['section'], 
                        capitulos_con_interpretaciones['interpretation'][cap]['paragraph']):
        markdown_text += f"### {sec}\n{par}\n\n"

    # Agregar el contenido Markdown como una sección al PDF
    pdf.add_section(
        Section(
            text=markdown_text,
        )
    )

pdf.save("salida.pdf")